# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [21]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key


In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("../output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,aswan,24.0934,32.9070,40.02,10,4,5.22,EG,1718821743
1,1,colonia,50.9333,6.9500,20.87,62,20,2.06,DE,1718821743
2,2,wodonga,-36.1167,146.8833,0.71,95,3,2.02,AU,1718821743
3,3,stanley,54.8680,-1.6985,18.07,60,20,5.66,GB,1718821548
4,4,longyearbyen,78.2186,15.6401,8.91,40,75,9.26,SJ,1718821744


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [4]:

# Configure the map plot
map_plot_1 = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    size = "Humidity",
    scale = 1,
    color = "City"
)

# Display the map
map_plot_1

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [7]:
# Narrow down cities that fit criteria and drop any results with null values
# low humidity is my ideal location (less than 50%)
ideal_city = city_data_df.loc[city_data_df["Humidity"]< 50]


# Drop any rows with null values
ideal_city = ideal_city.dropna()

# Display sample data
ideal_city


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,aswan,24.0934,32.9070,40.02,10,4,5.22,EG,1718821743
4,4,longyearbyen,78.2186,15.6401,8.91,40,75,9.26,SJ,1718821744
9,9,great falls,47.5002,-111.3008,16.66,47,20,2.57,US,1718821745
17,17,illizi,26.4833,8.4667,39.89,9,0,5.15,DZ,1718821747
21,21,tongliao,43.6125,122.2653,22.80,49,84,7.07,CN,1718821748
...,...,...,...,...,...,...,...,...,...,...
535,535,gurupi,-11.7292,-49.0686,33.88,28,0,2.62,BR,1718821881
544,544,saint-louis,16.3333,-15.0000,36.80,42,100,10.11,SN,1718821884
548,548,wad az zaki,14.4619,32.2065,35.75,32,0,2.77,SD,1718821885
554,554,mweka,-4.8500,21.5667,23.72,40,2,0.74,CD,1718821886


### Step 3: Create a new DataFrame called `hotel_df`.

In [9]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_city.filter(["City", "Country", "Lat", "Lng", "Humidity"])

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df.head(10)

,City,Country,Lat,Lng,Humidity,Hotel Name
0,aswan,EG,24.0934,32.9070,10,
4,longyearbyen,SJ,78.2186,15.6401,40,
9,great falls,US,47.5002,-111.3008,47,
17,illizi,DZ,26.4833,8.4667,9,
21,tongliao,CN,43.6125,122.2653,49,
25,atar,MR,20.5169,-13.0499,13,
44,kuruman,ZA,-27.4524,23.4325,30,
55,ataq,YE,14.5377,46.8319,41,
69,aripuana,BR,-9.1667,-60.6333,26,
72,cabo san lucas,MX,22.8909,-109.9124,27,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [16]:
# Set parameters to search for a hotel
radius = 10000
categories = "accommodation.hotel"
filters = ""
bias = ""
limit = 1

params = {
    "categories":categories,
    "filter":filters,
    "limit": limit,
    "bias": bias,
    "apiKey": geoapify_key    
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = hotel_df.at[index,"Lat"]
    longtitude = hotel_df.at[index,"Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f'circle:{longtitude},{latitude},{radius}'
    params["bias"] = f'proximity:{longtitude},{latitude}'
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
aswan - nearest hotel: Oscar Hotel
longyearbyen - nearest hotel: The Vault
great falls - nearest hotel: La Quinta Inn & Suites
illizi - nearest hotel: بوناقة
tongliao - nearest hotel: No hotel found
atar - nearest hotel: Residence Egueil
kuruman - nearest hotel: Palermo Inn
ataq - nearest hotel: الشارقة بلازا
aripuana - nearest hotel: No hotel found
cabo san lucas - nearest hotel: Comfort Rooms
zuni pueblo - nearest hotel: No hotel found
whitehorse - nearest hotel: Town & Mountain Hotel
yellowknife - nearest hotel: No hotel found
buckeye - nearest hotel: Days Inn
bukama - nearest hotel: Mission catholique
shache - nearest hotel: 莎車新盛賓舘
ayorou - nearest hotel: Hotel Aminokal
sidi amrane - nearest hotel: Darna
zarqa - nearest hotel: الدار
at taj - nearest hotel: No hotel found
labrador city - nearest hotel: Carol Inn
mandaguari - nearest hotel: No hotel found
allanridge - nearest hotel: No hotel found
tulsipur - nearest hotel: Ekanta Hotel
tindouf - nearest hotel: م

,City,Country,Lat,Lng,Humidity,Hotel Name
0,aswan,EG,24.0934,32.9070,10,Oscar Hotel
4,longyearbyen,SJ,78.2186,15.6401,40,The Vault
9,great falls,US,47.5002,-111.3008,47,La Quinta Inn & Suites
17,illizi,DZ,26.4833,8.4667,9,بوناقة
21,tongliao,CN,43.6125,122.2653,49,No hotel found
...,...,...,...,...,...,...
535,gurupi,BR,-11.7292,-49.0686,28,Via Norte
544,saint-louis,SN,16.3333,-15.0000,42,No hotel found
548,wad az zaki,SD,14.4619,32.2065,32,No hotel found
554,mweka,CD,-4.8500,21.5667,40,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [18]:
#clean up dataframe to not include cities with no hotels in the radius
clean_hotel = hotel_df.drop(hotel_df[hotel_df['Hotel Name']=="No hotel found"].index)
clean_hotel

,City,Country,Lat,Lng,Humidity,Hotel Name
0,aswan,EG,24.0934,32.9070,10,Oscar Hotel
4,longyearbyen,SJ,78.2186,15.6401,40,The Vault
9,great falls,US,47.5002,-111.3008,47,La Quinta Inn & Suites
17,illizi,DZ,26.4833,8.4667,9,بوناقة
25,atar,MR,20.5169,-13.0499,13,Residence Egueil
...,...,...,...,...,...,...
504,falun,SE,60.6036,15.6260,38,First Hotel Grand
506,alpena,US,45.0617,-83.4327,45,Holiday Inn Express & Suites
509,cafayate,AR,-26.0730,-65.9761,6,Il Cavallino
535,gurupi,BR,-11.7292,-49.0686,28,Via Norte


In [20]:

# Configure the map plot
map_plot_2= clean_hotel.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    scale = 1,
    color = "City",
    hover_cols = "all"
)


# Display the map
map_plot_2

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,index,Country,Humidity,Hotel Name)